
<img src="img/banner.png" alt="Exemplo de imagem" width=100%/>

# Este NoteBook Utiliza o Terminal de Negociações MetaTrader5 Para Coletar Dados Históricos de Ativos Financeiros e Cria Arquivo .CSV Com as Respectivas Cotações.
### Para sua utilização é necessário:
    1. Ter conta ativa em alguma corretora de valores que forneça sinal para o terminal de negociações 'MetaTrader 5'.
    2. Ter instalado o terminal de negociações 'MetaTrader 5'.
    3. Logar a conta da corretora ao terminal de negociações 'MetaTrader 5'
    
### Referência
    https://www.mql5.com/en/docs/python_metatrader5

# Definir Cores Para Células MarkDown

In [193]:
from IPython.core.display import HTML

HTML("""
<style>
    h1 {
        color: #6600cc;
    }
    h2,h3 {
    color: blue;
    }
    h4 {
    color: orange;
    }
    
</style>
""")

# Instanado Biblioteca

In [7]:
! pip install metatrader5

# Importando Bibliotecas

In [6]:
import pandas as pd
import numpy as np
import MetaTrader5 as mt5
import datetime
import warnings
warnings.filterwarnings('ignore')

# Testando Conexão Com MetaTrader5

In [235]:
def init_mt5(msg=True):
    path = r'C:\Program Files\MetaTrader 5'
    TERMINAL = r'\terminal64.exe'
    terminal = path + TERMINAL
    if not mt5.initialize(terminal):
        print("Falha ao inicializar o MetaTrader 5",mt5.last_error())
        quit()
    acct_info = mt5.account_info()._asdict()
    term_info = mt5.terminal_info()._asdict()
    if msg == True:
        print('========== Início das Informações de Login e Conta ==========')
        print('Login Account: ', acct_info['login'])
        print('Account Name: ', acct_info['name'])
        print('Login Server: ', acct_info['server'])
        print('Account Leverage: ', acct_info['leverage'])
        print('Account Balance: ', acct_info['balance'])
        print('Account Buying Power in Million: ', round(acct_info['leverage'] * acct_info['balance'] / 1_000_000, 3))
        print('Terminal: ', term_info['name'])
        print('Terminal Connected: ', term_info['connected'])
        print('Trade Allowed: ', term_info['trade_allowed'])
        print('Trade API Disabled: ', term_info['tradeapi_disabled'])
        print('========== Final das Informações de Login e Conta ==========')
init_mt5()

========== Início das Informações de Login e Conta ==========
Login Account: XXXXXX
Account Name: XXXXXX
Login Server: XPMT5-PROD
Account Leverage:  1
Account Balance:  9879768.79
Account Buying Power in Million:  9.88
Terminal:  MetaTrader 5
Terminal Connected:  True
Trade Allowed:  False
Trade API Disabled:  False
========== Final das Informações de Login e Conta ==========


# 0. Variáveis TimeFrame

    **Lista de Possíveis Variáveis Para Coletar Diferêntes Time Frames**
    mt5.TIMEFRAME_M1 = 1 minuto
    mt5.TIMEFRAME_M2 = 2 minutos
    mt5.TIMEFRAME_M3 = 3 minutos
    mt5.TIMEFRAME_M4 = 4 minutos
    mt5.TIMEFRAME_M5 = 5 minutos
    mt5.TIMEFRAME_M6 = 6 minutos
    mt5.TIMEFRAME_M10 = 10 minutos
    mt5.TIMEFRAME_M12 = 12 minutos
    mt5.TIMEFRAME_M15 = 15 minutos
    mt5.TIMEFRAME_M20 = 20 minutos
    mt5.TIMEFRAME_M30 = 30 minutos
    mt5.TIMEFRAME_H1 = 1 hora
    mt5.TIMEFRAME_H2 = 2 horas
    mt5.TIMEFRAME_H3 = 3 horas
    mt5.TIMEFRAME_H4 = 4 horas
    mt5.TIMEFRAME_H6 = 6 horas
    mt5.TIMEFRAME_H8 = 8 horas
    mt5.TIMEFRAME_H12 = 12 horas
    mt5.TIMEFRAME_D1 = 1 dia
    mt5.TIMEFRAME_W1 = 1 semana
    mt5.TIMEFRAME_MN1 = 1 mês

# 1. Coletando OHLC de Um Único Ativo
### Legenda
    O = Open (Preço Inicial Do Ativo Durante o TimeFrame Selecionado)
    H = High (Preço Máximo Do Ativo Durante o TimeFrame Selecionado)
    L = Low (Preço de Mínimo Do Ativo Durante o TimeFrame Selecionado)
    C = Close (Preço Final Do Ativo Durante o TimeFrame Selecionado)

#### Variáveis

In [221]:
# Ativo
ativo = ('wege3').upper()

# Definir Time Frame das Cotações
tf = mt5.TIMEFRAME_D1 
    
# Definir Número de Cotações
cotacoes = 100 

In [222]:
# Coletando Cotações
data = pd.DataFrame(mt5.copy_rates_from_pos(ativo, tf, 0, cotacoes))

# Excluir Coluna Indesejada
del data['spread']

# Definir Colunas 'time' Como Index
data.set_index('time', inplace=True)

# Converter Formato Index Para DateTime
data.index = pd.to_datetime(data.index, unit='s').date

# Copiar DataFrame
df = data.copy()

# Verificar Dados Coletados
df.head(2)

,open,high,low,close,tick_volume,real_volume
2022-11-17,38.00,39.44,37.22,39.23,27598,8238900
2022-11-18,39.24,39.96,38.79,38.97,20554,6670400


### Salvar Dados em Formato .CSV

In [223]:
# Copiar DataFrame
df_csv = df.copy()

# Substitui todos os pontos por vírgulas
df_csv = df_csv.round(2)
df_csv = df_csv.applymap(lambda x: str(x).replace('.', ','))

# Salvar DataFrame em Formato .CSV
df_csv.to_csv('Single_OHLC.csv', index=True)

# Conferir Arquivo Salvo
pd.read_csv('Single_OHLC.csv', index_col=0).tail(2)

,open,high,low,close,tick_volume,real_volume
2023-04-10,"38,51","38,76","38,17","38,33",8758,4469500
2023-04-11,"38,65","39,33","38,64","39,04",6551,3798200


# 2. Coletando Fechamento de Um Único Ativo
### Comumente apenas o preço de fechamento (Close) é utilizado para analises.

#### Variáveis

In [224]:
# Ativo
ativo = ('wege3').upper()

# Definir Time Frame das Cotações
tf = mt5.TIMEFRAME_D1 

# Definir Número de Cotações
cotacoes = 100 

In [225]:
# Coletando Cotações
data = pd.DataFrame(mt5.copy_rates_from_pos(ativo, tf, 0, cotacoes))[['time','close']]

# Definir Colunas 'time' Como Index
data.set_index('time', inplace=True)

# Converter Formato Index Para DateTime
data.index = pd.to_datetime(data.index, unit='s').date

# Renomerar Colunas
data = data.rename({'close': ativo}, axis=1)

# Copiar DataFrame
df = data.copy()

# Verificar Dados Coletados
df.head(2)

,WEGE3
2022-11-17,39.23
2022-11-18,38.97


### Salvar Dados em Formato .CSV

In [226]:
# Copiar DataFrame
df_csv = df.copy()

# Substitui todos os pontos por vírgulas
df_csv = df_csv.round(2)
df_csv = df_csv.applymap(lambda x: str(x).replace('.', ','))

# Salvar DataFrame em Formato .CSV
df_csv.to_csv('Single_Close.csv', index=True)

# Conferir Arquivo Salvo
pd.read_csv('Single_Close.csv', index_col=0).tail(2)

,WEGE3
2023-04-10,"38,33"
2023-04-11,"39,04"


# 3. Coletando Fechamento de Múltiplos Ativos

#### Variáveis

In [227]:
# Lista de Ativos
ativos = ['COGN3','VALE3','WEGE3','cogn3']

# Definir Time Frame das Cotações
tf = mt5.TIMEFRAME_D1 

# Definir Número de Cotações
cotacoes = 100 

In [228]:
# Coletando Loop Cotações
df = pd.DataFrame()
for i in ativos:
    
    # Transformar o Código do Ativo Em Caixa-Alta
    i = i.upper()
    
    # Coletar Dados
    data = pd.DataFrame(mt5.copy_rates_from_pos(i.upper(), tf, 0, cotacoes))[['time','close']]
    
    # Definir Colunas 'time' Como Index
    data.set_index('time', inplace=True)
    
    # Converter Formato Index Para DateTime
    data.index = pd.to_datetime(data.index, unit='s').date
    
    # Renomerar Colunas
    data = data.rename({'close': i}, axis=1)
    
    # Concatenar Cotações Coletadas de Diferentes Ativos
    df = pd.concat([df, data], axis=1)

# Verificar Dados Coletados
df.head(2)

,COGN3,VALE3,WEGE3,COGN3
2022-11-17,2.35,81.08,39.23,2.35
2022-11-18,2.26,78.86,38.97,2.26


### Salvar Dados em Formato .CSV

In [229]:
# Copiar DataFrame
df_csv = df.copy()

# Substitui todos os pontos por vírgulas
df_csv = df_csv.round(2)
df_csv = df_csv.applymap(lambda x: str(x).replace('.', ','))

# Salvar DataFrame em Formato .CSV
df_csv.to_csv('Multi_Close.csv', index=True)

# Conferir Arquivo Salvo
pd.read_csv('Multi_Close.csv', index_col=0).tail(2)

,COGN3,VALE3,WEGE3,COGN3.1
2023-04-10,"1,77","78,16","38,33","1,77"
2023-04-11,"1,93","82,84","39,04","1,93"


# 4. Coletando OHLC de Múltiplos Ativos
### Legenda
    O = Open (Preço Inicial Do Ativo Durante o TimeFrame Selecionado)
    H = High (Preço Máximo Do Ativo Durante o TimeFrame Selecionado)
    L = Low (Preço de Mínimo Do Ativo Durante o TimeFrame Selecionado)
    C = Close (Preço Final Do Ativo Durante o TimeFrame Selecionado)

#### Variáveis

In [230]:
# Lista de Ativos
ativos = ['COGN3','VALE3','WEGE3','cogn3']

# Definir Time Frame das Cotações
tf = mt5.TIMEFRAME_D1 

# Definir Número de Cotações
cotacoes = 100 

### Coletar Dados

In [231]:
# Loop de Cotações
df = pd.DataFrame()
for i in ativos:
    
    # Transformar o Código do Ativo Em Caixa-Alta
    i = i.upper()
    
    # Coletar Dados
    data = pd.DataFrame(mt5.copy_rates_from_pos(i.upper(), tf, 0, cotacoes)).iloc[:,[0,1,2,3,4]]
    
    # Definir Colunas 'time' Como Index
    data.set_index('time', inplace=True)
    
    # Converter Formato Index Para DateTime
    data.index = pd.to_datetime(data.index, unit='s').date
    
    # Renomerar Colunas
    data = data.rename({'open': i+'_O', 
                        'high': i+'_H',
                        'low': i+'_L',
                        'close': i+'_C'
                       }, axis=1)
    
    # Concatenar Cotações Coletadas de Diferentes Ativos
    df = pd.concat([df, data], axis=1)
    
# Verificar Dados Coletados 
df.head(2)

,COGN3_O,COGN3_H,COGN3_L,COGN3_C,VALE3_O,VALE3_H,VALE3_L,VALE3_C,WEGE3_O,WEGE3_H,WEGE3_L,WEGE3_C,COGN3_O,COGN3_H,COGN3_L,COGN3_C
2022-11-17,2.36,2.38,2.25,2.35,78.9,81.26,78.78,81.08,38.00,39.44,37.22,39.23,2.36,2.38,2.25,2.35
2022-11-18,2.40,2.45,2.25,2.26,82.3,82.43,78.74,78.86,39.24,39.96,38.79,38.97,2.40,2.45,2.25,2.26


### Salvar Dados em Formato .CSV

In [232]:
# Copiar DataFrame
df_csv = df.copy()

# Substitui todos os pontos por vírgulas
df_csv = df_csv.round(2)
df_csv = df_csv.applymap(lambda x: str(x).replace('.', ','))

# Salvar DataFrame em Formato .CSV
df_csv.to_csv('Multi_OHLC.csv', index=True)

# Conferir Arquivo Salvo
pd.read_csv('Multi_OHLC.csv', index_col=0).tail(2)

,COGN3_O,COGN3_H,COGN3_L,COGN3_C,VALE3_O,VALE3_H,VALE3_L,VALE3_C,WEGE3_O,WEGE3_H,WEGE3_L,WEGE3_C,COGN3_O.1,COGN3_H.1,COGN3_L.1,COGN3_C.1
2023-04-10,"1,79","1,8","1,76","1,77","76,75","78,26","76,7","78,16","38,51","38,76","38,17","38,33","1,79","1,8","1,76","1,77"
2023-04-11,"1,8","1,94","1,8","1,93","80,5","82,9","80,06","82,84","38,65","39,33","38,64","39,04","1,8","1,94","1,8","1,93"


In [233]:
# Encerrar Conexão Com o Terminal de Negociações MetaTrader5
mt5.shutdown()

True